In [1]:
!pip3 install "gym[atari]"
import gym
import sys
from PIL import Image, ImageDraw
#sys.path.append('./drive/My Drive/Colab Notebooks/neuroevolution_toolbox/')
import numpy as np
import pandas as pd
import neural_tape_controller
#Положительные числа - положительные награды.
import tasks 
import optimize
import pickle5 as pickle
root_dir = './'

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
# HIDE OUTPUT
!wget http://www.atarimania.com/roms/Roms.rar 
!unrar x -o+ /content/Roms.rar >/dev/nul
!python -m atari_py.import_roms /content/ROMS >/dev/nul

In [3]:
def query_environment(name):
    env = gym.make(name)
    spec = gym.spec(name)
    print(f"Action Space: {env.action_space}")
    print(f"Observation Space: {env.observation_space}")
    print(f"Max Episode Steps: {spec.max_episode_steps}")
    print(f"Nondeterministic: {spec.nondeterministic}")
    print(f"Reward Range: {env.reward_range}")
    print(f"Reward Threshold: {spec.reward_threshold}")
query_environment("ALE/Breakout-v5")

Action Space: Discrete(4)
Observation Space: Box(0, 255, (210, 160, 3), uint8)
Max Episode Steps: 27000
Nondeterministic: False
Reward Range: (-inf, inf)
Reward Threshold: None


In [4]:
def encode_simple(X):
  #как сделать простое кодирование? Ну давай разобьём картинку двумя разными сетками и измерим яркость
  #на входе одиночная картинка
  y_lst = []
  sz = np.shape(X)
  count_squares = 7
  for i in range(count_squares):
    for j in range(count_squares):
      x = X[int(i*sz[0]/count_squares):int((i+1)*sz[0]/count_squares), int(j*sz[1]/count_squares):int((j+1)*sz[1]/count_squares)]
      y_lst.append(np.nanmean(x))

  count_squares = 3
  for i in range(count_squares):
    for j in range(count_squares):
      x = X[int(i*sz[0]/count_squares):int((i+1)*sz[0]/count_squares), int(j*sz[1]/count_squares):int((j+1)*sz[1]/count_squares)]
      y_lst.append(np.nanmean(x))

  count_squares = 2
  for i in range(count_squares):
    for j in range(count_squares):
      x = X[int(i*sz[0]/count_squares):int((i+1)*sz[0]/count_squares), int(j*sz[1]/count_squares):int((j+1)*sz[1]/count_squares)]
      y_lst.append(np.nanmean(x))
  #размер: 62
  return np.array(y_lst)

In [5]:
n_actions = 4
embed_size = 62#это эмбеддинг картинки
nt = neural_tape_controller.nt_controller(input_size=embed_size,output_size=n_actions)
try:
    with open(root_dir+f'./genoms/best_genoms_breakout.pkl', 'rb') as f:
        genom = pickle.load(f)
        genom = genom[-1]
    print('loaded successfully')
except Exception:
    genom = nt.nn.disassemble_genom()

In [11]:
def test_atary(genom, draw=False,game_name="ALE/Breakout-v5",seed=0):
  if draw:
    globals()['video'] = []

  env = gym.make(game_name)
  env.reset()
  n_actions = env.action_space.n
  state_dim = env.observation_space.shape#здесь картинка
  #print('n_actions',n_actions,'state_dim',state_dim)
  #1/0
  state_dim = embed_size#это эмбеддинг картинки
  controller = neural_tape_controller.nt_controller(tacts=1,genom=np.array(genom),input_size=state_dim,output_size=n_actions) 
  out_tape = np.zeros(30)
  reward_sum = 0

  #seed=1
  np.random.seed(seed)
  env.seed(seed)
  for i in range(400):
      action = np.ravel(out_tape)
      action += np.random.rand(len(action))*np.std(action)*0.05#игра детерминистическая, иначе рандома не будет
      #print('action',int(np.argmax(action)),action)
      #исполнить env
      state, reward, done,_ = env.step(int(np.argmax(action)))
      t=pd.Timestamp.now()
      #state = encode_mobnet(np.array([state]))
      state = encode_simple(state)
      reward_sum += reward
      if np.shape(state)[0]>1:
          shp = np.shape(state)
          state = np.reshape(state,[1,shp[0]])
      out_tape = controller.act(state,reward,done)
      if draw:
        globals()['video'].append(Image.fromarray(env.render("rgb_array")))
  return reward_sum

In [12]:
def multy_test(genom):
  game_name = "ALE/Breakout-v5"
  #один энвайронмент, разные сиды
  q_arr = []
  for seed in range(3):
      q_arr.append(test_atary(genom, draw=False,game_name=game_name,seed=seed))
  return np.mean(q_arr) - 0.000000001*np.sum(genom**2)

In [10]:
%%time
q=test_atary(genom,game_name="ALE/Breakout-v5", draw=False)
print('q=',q)

q= 0.0
CPU times: user 5.54 s, sys: 12.2 s, total: 17.8 s
Wall time: 20.5 s


In [13]:
%%time
q=multy_test(genom)
print('q=',q)

q= -6.75390625e-09
CPU times: user 13.3 s, sys: 26.6 s, total: 39.9 s
Wall time: 53.4 s


In [15]:
%%time
q=test_atary(genom, draw=True,seed=0)
print('q=',q)

q= 0.0
CPU times: user 3.92 s, sys: 6.93 s, total: 10.8 s
Wall time: 22.2 s


In [16]:
video[0].save(
    root_dir+'./out_videos/breakout_neuroevol.gif',
    save_all=True,
    append_images=video[1:], 
    optimize=True,
    duration=100,
    loop=0
)

In [ ]:
print(pd.Timestamp.now())

In [14]:
print(1)
opt = optimize.optimizer(multy_test, genom_size=len(genom),parallel_cores=12)
try:
    with open(root_dir+f'best_genoms.pkl', 'rb') as f:
        opt.best_genoms = pickle.load(f)
    print('loaded successfully')
except Exception:
    pass
opt.function(opt.best_genoms[-1])

for i in range(1000):
    print('opt#',i)
    opt.optimize()
    with open(root_dir+f'best_genoms.pkl', 'wb') as f:
        pickle.dump(opt.best_genoms,f,protocol=pickle.HIGHEST_PROTOCOL)
        print('WRITTEN')

1


/home/nikita/neuroevolution-toolbox/nnet.py:316: RuntimeWarning: overflow encountered in add
  self.belts[layer['belt_name']][:,:min_len] += in_data[:,:min_len]
/home/nikita/neuroevolution-toolbox/nnet.py:324: RuntimeWarning: overflow encountered in multiply
  k_add = np.arctan(k_amplif*self.belts[layer['belt_name']][:,:min_len]/threshold)*threshold


opt# 0
scores for optimizers augmented [1.e+10 1.e+10 1.e+10 1.e+10 1.e+10 1.e+10 1.e+10 1.e+10]
chosen evol_wide previous_result: nan per tacts: nan


/home/nikita/neuroevolution-toolbox/optimize.py:36: RuntimeWarning: Mean of empty slice
  mx.append(np.nanmean(self.history_gain[opt_name])-time_penalty*np.nanmean(self.history_time[opt_name]))
/home/nikita/.local/lib/python3/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


iteration 0 y= [5.66664426 4.33331092 2.99997765 2.99997756]
iteration 1 y= [5.66664426 4.33331092 3.66664425 3.33331091]
iteration 2 y= [5.66664426 4.33331092 4.33331091 3.66664425]
iteration 3 y= [5.66664426 4.33331092 4.33331091 3.66664425]
iteration final y= [5.66664426 5.33331093 5.33331092 4.33331092]
result evol_wide previous_gain: nan per tacts: nan duration 0 days 00:45:32.911093
WRITTEN
opt# 1
scores for optimizers augmented [-1.e-01  1.e+10  1.e+10  1.e+10  1.e+10  1.e+10  1.e+10  1.e+10]
chosen evol_mid_chaos previous_result: nan per tacts: nan
iteration 0 y= [5.66664426 4.33331095 2.66664432]
iteration 1 y= [5.66664426 5.6666441  4.33331095]
iteration final y= [6.99997728 6.99997727 5.66664426]
result evol_mid_chaos previous_gain: nan per tacts: nan duration 0 days 00:05:57.931042
WRITTEN
opt# 2
scores for optimizers augmented [-1.00000000e-01  1.31233301e+00  1.00000000e+10  1.00000000e+10
  1.00000000e+10  1.00000000e+10  1.00000000e+10  1.00000000e+10]
chosen gradient_w

KeyboardInterrupt: 